In [1]:
import cv2
import os,sys,glob
import re
import pickle
import pandas as pd
import numpy as np
from fun_1_glob_crop_downsample_concatenate import index_videos,crop_downsample_concatenate

In [3]:
dirs = glob.glob(r"F:\miniscope\*")


In [5]:
concatenate_files_1 = glob.glob(r"F:\miniscope\*08*\206551")
concatenate_files_2 = glob.glob(r"F:\miniscope\*09*\206551")
concatenate_files = concatenate_files_1 + concatenate_files_2
concatenate_files = [concatenate_files[11]]
concatenate_files

['F:\\miniscope\\20200801\\2020061',
 'F:\\miniscope\\20200803\\2020061',
 'F:\\miniscope\\20200804\\2020061',
 'F:\\miniscope\\20200805\\2020061',
 'F:\\miniscope\\20200806\\2020061',
 'F:\\miniscope\\20200807\\2020061',
 'F:\\miniscope\\20200810\\2020061',
 'F:\\miniscope\\20200811\\2020061',
 'F:\\miniscope\\20200812\\2020061',
 'F:\\miniscope\\20200814\\2020061',
 'F:\\miniscope\\20200816\\2020061',
 'F:\\miniscope\\20200818\\2020061',
 'F:\\miniscope\\20200819\\2020061',
 'F:\\miniscope\\20200820\\2020061',
 'F:\\miniscope\\20200821\\2020061',
 'F:\\miniscope\\20200823\\2020061',
 'F:\\miniscope\\20200824\\2020061',
 'F:\\miniscope\\20200825\\2020061',
 'F:\\miniscope\\20200826\\2020061',
 'F:\\miniscope\\20200828\\2020061']

In [4]:
bug_msFileList5 = batch_concatenate(concatenate_files=concatenate_files
                  ,rawdataDir=r"F:\miniscope"
                  ,resultDir=r"\\10.10.47.163\Data_archive\qiushou\miniscope_result"
                  ,spatial_downsampling=2
                  ,temporal_downsampling=1
                  ,video_process=True)

20200824 206551
\\10.10.47.163\Data_archive\qiushou\miniscope_result\Results_206551\crop_param.pkl
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam1.avi
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam2.avi
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam3.avi
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam4.avi
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam5.avi
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam6.avi
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam7.avi
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam8.avi
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam9.avi
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam10.avi
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam11.avi
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam12.avi
Concatenating F:\miniscope\20200824\206551\H18_M10_S49\msCam13.avi
Concatenating F:\miniscope\20200824\206

t:   0%|                                                                 | 57/39946 [00:00<02:32, 260.97it/s, now=None]


Moviepy - Writing video \\10.10.47.163\Data_archive\qiushou\miniscope_result\Results_206551\20200923_111509_30fps_20200824\msCam_concat.avi



Moviepy - Done !
Moviepy - video ready \\10.10.47.163\Data_archive\qiushou\miniscope_result\Results_206551\20200923_111509_30fps_20200824\msCam_concat.avi
\\10.10.47.163\Data_archive\qiushou\miniscope_result\Results_206551\20200923_111509_30fps_20200824\ms_ts.pkl
camNum in miniscope is 0
concatenated video and timestamps have the same frames
concatenated timestamp of miniscope_video is located at \\10.10.47.163\Data_archive\qiushou\miniscope_result\Results_206551\20200923_111509_30fps_20200824\ms_ts.pkl
done


In [64]:
bug_msFileList5

[]

In [23]:
bug_msFileList

[]

In [3]:
def batch_concatenate(concatenate_files
                      ,rawdataDir
                      ,resultDir
                      ,spatial_downsampling=2
                      ,temporal_downsampling=3
                     ,video_process=True):
    bug_msFileList=[]
    for file in concatenate_files:
        date = re.findall('\d{8}',file)[0]
        mouse_id = re.findall('\d{8}\\\\(\d+)',file)[0]
        print(date,mouse_id)
        msFileList,tsFileList = index_videos(animal_id=mouse_id
                                             ,date=date
                                             ,recorded_method="miniscope"
                                             ,rawdataDir=rawdataDir)
#         print(tsFileList)
        try:
            crop_downsample_concatenate(animal_id=mouse_id
                                        ,msFileList=msFileList
                                        ,tsFileList=tsFileList
                                        ,note="_30fps_"+date
                                        ,spatial_downsampling=spatial_downsampling
                                        ,temporal_downsampling=temporal_downsampling
                                        ,resultDir=resultDir
                                        ,video_process=video_process)
            print("done")
        except Exception as e:
            print(e)
            bug_msFileList.append((mouse_id,date))
            continue
    return bug_msFileList

In [ ]:
def pkl2mat2(ms_mat_path):
    """
    将多个ms_ts.pkl写入到一个ms.mat中
    考虑多天合并跑的情况
    """

    result = load_mat(ms_mat_path)

    pkl_pathes = glob.glob(os.path.join(os.path.dirname(ms_mat_path),"*\ms_ts.pkl"))

    mat_path = os.path.join(os.path.dirname(ms_mat_path),"ms2.mat")
    ms_ts_all=[]
    for pkl_path in pkl_pathes:
        with open(pkl_path,"rb") as f:
            ms_ts = pickle.load(f)
        for temp in ms_ts:
            ms_ts_all.append(np.array(temp))
    result["ms"]["ms_ts"]=np.array(ms_ts_all)
    savemat(mat_path,result)
    print("save mat %s"%mat_path)